In [13]:
import numpy as np
import pandas as pd

## 데이터 로드

In [14]:
data=pd.read_csv("/content/drive/MyDrive/ChatbotData .csv")

In [15]:
data_np=data.to_numpy()

In [16]:
#Q_data는 토큰화 하여 분석할 데이터
#L_dataa는 라벨 데이터
Q_data=[]
L_data=[]
for index,data_ in enumerate(data_np):
    Q_data.append(data_[0])
    if data_[2]==0:
        L_data.append("일상")
    elif data_[2]==1:
        L_data.append("부정")
    else:
        L_data.append("긍정")

In [17]:
Q_data=np.array(Q_data)
L_data=np.array(L_data)

In [18]:
print(Q_data[:5])
print(L_data[:5])

['12시 땡!' '1지망 학교 떨어졌어' '3박4일 놀러가고 싶다' '3박4일 정도 놀러가고 싶다' 'PPL 심하네']
['일상' '일상' '일상' '일상' '일상']


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(Q_data, L_data, test_size=0.33) #훈련셋과 테스트셋으로 분리

In [20]:
print(type(y_train))

<class 'numpy.ndarray'>


## 꼬꼬마를 이용한 형태소 분석

In [21]:
!pip install konlpy #--> 본인 환경에서 konlpy가 설치 안되어있을 때 주석 해제

     |████████████████████████████████| 19.4MB 8.3MB/s 
     |████████████████████████████████| 460kB 50.3MB/s 
     |████████████████████████████████| 92kB 14.3MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Kkma  
kkma=Kkma()  

In [23]:
kkma_data_train=[] # 각 데이터셋 별로 형태소 분석 수행
for index, data_ in enumerate(X_train):
    kkma_data_train.append(kkma.morphs(data_))
    if index%800 ==0:
        print(kkma.morphs(data_))
kkma_data_test=[]
for index, data_ in enumerate(X_test):
    kkma_data_test.append(kkma.morphs(data_))

['또', '한번', '의', '무너지', 'ㅁ']
['이별', '통보', '후', '일주일', '이', '네']
['동호회', '약속', '있', '어', '.']
['이제', '나', '자신', '으로', '돌아오', '아', '아름답', '어', '지', '고', '싶', '어', '.']
['마스크', '팩', '대량', '구매', '하', '었', '어']
['두', '번째', '이별', '이야기']
['좋', '은', '사람', '들', '은', '다', '어디', '가', '었', '을까']
['백만', '번', '쯤', '듣', '을', '때', '즈음', '이', '면', '웃', '으며', '다가오', 'ㄹ까']
['영양제', '좀', '먹', '을까']
['상처', '주지', '말', 'ㅂ시다', '.']


In [24]:
kkma_data_T=sum(kkma_data_train,[])

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv=TfidfVectorizer().fit(kkma_data_T)
print(tfidfv.vocabulary_)
print(len(tfidfv.vocabulary_))

{'한번': 3045, '무너지': 1076, '하나': 2999, '완전': 2058, '재미': 2410, '거든': 156, '방법': 1195, '친구': 2815, '비교': 1369, '작아지': 2370, '사랑': 1425, '별거': 1249, '관심': 290, '여자애': 1964, '한테': 3058, '의미': 2203, '선물': 1534, '은데': 2188, '무엇': 1096, '좋아하': 2542, 'ㄹ까요': 33, '관계': 286, '계속': 219, '애매': 1847, '현실': 3113, '에서': 1940, '도망치': 727, '용기': 2090, '필요': 2991, '아프': 1812, '지만': 2622, '성형': 1557, 'ㄹ까': 32, '힘들': 3204, '자꾸': 2342, '나요': 482, '남자': 506, '호감': 3117, '으면': 2180, '연락': 1977, '안해': 1832, '반납': 1162, '뚱뚱': 854, '한데': 3040, '에게': 1935, '어도': 1875, '말하': 957, '귀엽': 330, '이별': 2239, '20': 7, '사람': 1424, '이랑': 2219, '일어나': 2298, '자마자': 2349, '우울': 2103, '한숨': 3050, '뭐하': 1117, '물어보': 1114, '고맙': 234, '어디': 1877, '까지': 428, '가능': 58, '아무': 1784, '처럼': 2729, '만나': 934, '는데': 601, '지금': 2605, '불타오르': 1351, '면허': 1027, '으로': 2178, '는다는': 600, '안되': 1822, '데이트': 722, '비용': 1379, '너무': 538, '많이': 947, '나오': 481, '정리': 2473, '뱃살': 1220, '빵빵': 1401, '정말': 2474, '짝사랑': 2678, '니까': 606, '감정': 127, '기복': 3

In [26]:
model_train=[]
for index,data_ in enumerate(kkma_data_train):
    # tmp=np.zeros((90,3257),)
    #print(data_)
    tmp=tfidfv.transform(data_).toarray()
    #print(tmp,"check")
    #print(np.array(tmp).shape)
    tmp2=[3257,]
    for index2,data_2 in enumerate(tmp):
        tmp2+=data_2
    #print(np.array(tmp2).shape)
    model_train.append(tmp2)

In [30]:
model_test=[]
for index,data_ in enumerate(kkma_data_test):
    # tmp=np.zeros((90,3257),)
    #print(data_)
    tmp=tfidfv.transform(data_).toarray()
    #print(tmp,"check"")
    #print(np.array(tmp).shape)
    tmp2=[3257,]
    for index2,data_2 in enumerate(tmp):
        tmp2+=data_2
    #print(np.array(tmp2).shape)
    model_test.append(tmp2)

In [31]:
model_test=np.array(model_test) #머신러닝 분류기 모델의 입력으로 사용하기 위하여 넘파이 배열로 변환
y_test=np.array(y_test)

In [28]:
print(model_train.shape)
print(y_train.shape)

(7921, 3207)
(7921,)


## 머신러닝 분류기를 이용한 분류

In [29]:
from sklearn.svm import SVC
svm=SVC(kernel="rbf",C=1,gamma=1)
svm.fit(model_train,y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [32]:
result=svm.predict(model_test)

In [33]:
print(result[:5])
print(y_test[:5])

['부정' '긍정' '긍정' '일상' '긍정']
['부정' '긍정' '긍정' '일상' '긍정']


In [34]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, result))

0.6957970271655561
